In [1]:
from sportsipy.nfl.boxscore import Boxscore, BoxscorePlayer, Boxscores
from sportsipy.nfl.roster import Roster, Player, AbstractPlayer
from sportsipy.nfl.schedule import Game, Schedule
from sportsipy.nfl.teams import Team, Teams
from sportsipy.nfl.constants import *

import pandas as pd
pd.set_option('display.max_columns', None)

year = '2020'
team = 'GNB'
player_id = 'AdamDa01'
player_name = 'Davante Adams'
player_data = Player(player_id).dataframe.to_html()

# Steps to setup `get_data.py` file to get all information into DataFrame

1) Loop through Boxscores(`week=1`, `year={looped_number}`, `end_week=22`)
    - Returns Dictionary that will contain all game boxscores as strings like: `202009130min`
    - Get all these strings into a list to also loop through

[Sportsipy docs](https://sportsreference.readthedocs.io/en/stable/nfl.html#module-sportsipy.nfl.boxscore)

[Sportsipy Github](https://github.com/roclark/sportsipy/tree/ea0043747015209550abeee15df75914a58fe40b/sportsipy/nfl)

# Boxscore

* Boxscore(`AbstractPlayer`)

In [4]:
boxscores = Boxscores(1, 2001, end_week=20).games
# boxscores_dict = boxscores.games
boxscores

{'1-2001': [{'boxscore': '200109090kan',
   'away_name': 'Oakland Raiders',
   'away_abbr': 'rai',
   'away_score': 27,
   'home_name': 'Kansas City Chiefs',
   'home_abbr': 'kan',
   'home_score': 24,
   'winning_name': 'Oakland Raiders',
   'winning_abbr': 'rai',
   'losing_name': 'Kansas City Chiefs',
   'losing_abbr': 'kan'},
  {'boxscore': '200109090gnb',
   'away_name': 'Detroit Lions',
   'away_abbr': 'det',
   'away_score': 6,
   'home_name': 'Green Bay Packers',
   'home_abbr': 'gnb',
   'home_score': 28,
   'winning_name': 'Green Bay Packers',
   'winning_abbr': 'gnb',
   'losing_name': 'Detroit Lions',
   'losing_abbr': 'det'},
  {'boxscore': '200109090dal',
   'away_name': 'Tampa Bay Buccaneers',
   'away_abbr': 'tam',
   'away_score': 10,
   'home_name': 'Dallas Cowboys',
   'home_abbr': 'dal',
   'home_score': 6,
   'winning_name': 'Tampa Bay Buccaneers',
   'winning_abbr': 'tam',
   'losing_name': 'Dallas Cowboys',
   'losing_abbr': 'dal'},
  {'boxscore': '200109090min',

In [2]:
# Function to get long list of boxscore strings to be used like Boxscore('string') for each game to get boxscores

boxscore_strings = []
end_week = 21

# Getting all strings for boxscores in a list to be iterated through
for year in range(2020, 2021):
    print(year)
    try:
        boxscores_dict = Boxscores(1, year, end_week=end_week).games
    except:
        print(f'For year {year} it did not find {end_week} weeks, pulling {end_week-1} instead')
        boxscores_dict = Boxscores(1, year, end_week=end_week-1).games

    for idx, val in enumerate(boxscores_dict.keys()):
        for game in range(len(boxscores_dict[val])):
            boxscore_strings.append(boxscores_dict[val][game]['boxscore'])

2020


In [5]:
boxscore_strings[0]

'202009100kan'

In [12]:
# Function to gather all boxscore strings into single dataframe

df_boxscore = Boxscore(boxscore_strings[0]).dataframe

for game in boxscore_strings:
    print(f'Spinning up {game}')
    
    try:
        boxscore = Boxscore(game).dataframe
        df_boxscore = df_boxscore.append(boxscore)
    except:
        print(f'DAWWWWW {game} did not get a boxscore')
        continue

Spinning up 202009100kan
Spinning up 202009130buf
Spinning up 202009130atl
Spinning up 202009130was
Spinning up 202009130rav
Spinning up 202009130nwe
Spinning up 202009130min
Spinning up 202009130jax
Spinning up 202009130det
Spinning up 202009130car
Spinning up 202009130cin
Spinning up 202009130sfo
Spinning up 202009130nor
Spinning up 202009130ram
Spinning up 202009140nyg
Spinning up 202009140den
Spinning up 202009170cle
Spinning up 202009200clt
Spinning up 202009200chi
Spinning up 202009200tam
Spinning up 202009200pit
Spinning up 202009200phi
Spinning up 202009200oti
Spinning up 202009200nyj
Spinning up 202009200mia
Spinning up 202009200gnb
Spinning up 202009200dal
Spinning up 202009200crd
Spinning up 202009200sdg
Spinning up 202009200htx
Spinning up 202009200sea
Spinning up 202009210rai
Spinning up 202009240jax
Spinning up 202009270buf
Spinning up 202009270atl
Spinning up 202009270pit
Spinning up 202009270phi
Spinning up 202009270nyg
Spinning up 202009270nwe
Spinning up 202009270min


In [13]:
df_boxscore

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_under,roof,stadium,surface,time,vegas_line,weather,winner,winning_abbr,winning_name,won_toss
202009100kan,15895,21,1,1,0,0,1,242,32,20,1,253,5,20,22,2,118,10,4,25:13,4,360,1,37,11,"Thursday Sep 10, 2020",2020-09-10 20:20:00,2:53,28,1,1,0,0,0,203,32,24,3,211,1,34,34,1,166,13,7,34:47,1,369,0,5,8,HTX,Houston Texans,53.5 (over),Outdoors,Arrowhead Stadium,Astroturf,8:20pm,Kansas City Chiefs -9.5,"56 degrees, relative humidity 95%, wind 7 mph",Home,KAN,Kansas City Chiefs,Chiefs (deferred)
202009100kan,15895,21,1,1,0,0,1,242,32,20,1,253,5,20,22,2,118,10,4,25:13,4,360,1,37,11,"Thursday Sep 10, 2020",2020-09-10 20:20:00,2:53,28,1,1,0,0,0,203,32,24,3,211,1,34,34,1,166,13,7,34:47,1,369,0,5,8,HTX,Houston Texans,53.5 (over),Outdoors,Arrowhead Stadium,Astroturf,8:20pm,Kansas City Chiefs -9.5,"56 degrees, relative humidity 95%, wind 7 mph",Home,KAN,Kansas City Chiefs,Chiefs (deferred)
202009130buf,None,15,0,0,1,1,1,202,35,21,1,215,9,17,15,1,52,11,4,18:43,3,254,2,95,13,"Sunday Sep 13, 2020",2020-09-13 13:00:00,3:10,31,1,1,2,2,0,306,46,33,2,312,7,27,32,1,98,14,7,41:17,3,404,2,79,6,NYJ,New York Jets,39.5 (over),Outdoors,Bills Stadium,Astroturf,1:00pm,Buffalo Bills -6.5,"67 degrees, relative humidity 93%, wind 15 mph",Home,BUF,Buffalo Bills,Bills (deferred)
202009130atl,None,22,1,1,1,0,0,299,35,31,4,322,6,38,20,1,84,9,3,30:36,3,383,0,46,23,"Sunday Sep 13, 2020",2020-09-13 13:00:00,3:07,28,4,0,1,1,1,434,54,37,2,450,6,25,21,1,72,14,7,29:24,2,506,2,72,16,ATL,Atlanta Falcons,49.5 (over),Retractable Roof (Closed),Mercedes-Benz Stadium,Fieldturf,1:00pm,Atlanta Falcons -1.0,None,Away,SEA,Seattle Seahawks,Falcons
202009130was,None,19,2,0,3,1,2,208,42,24,2,270,3,17,17,0,57,14,5,30:39,8,265,3,20,62,"Sunday Sep 13, 2020",2020-09-13 13:00:00,3:15,18,2,1,2,0,0,159,31,17,1,178,7,27,36,2,80,18,5,29:21,3,239,0,55,19,PHI,Philadelphia Eagles,41.5 (over),Outdoors,FedExField,Grass,1:00pm,Philadelphia Eagles -5.5,"76 degrees, relative humidity 68%, wind 5 mph",Home,WAS,Washington Football Team,Eagles (deferred)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202101170kan,16730,21,3,3,1,1,1,196,37,23,1,204,3,17,22,1,112,13,6,29:23,1,308,2,25,8,"Sunday Jan 17, 2021",2021-01-17 15:05:00,3:01,24,2,2,0,0,1,315,38,27,1,321,7,22,24,1,123,10,5,30:37,1,438,1,55,6,CLE,Cleveland Browns,55.5 (under),Outdoors,Arrowhead Stadium,Astroturf,3:05pm,Kansas City Chiefs -7.5,"40 degrees, relative humidity 62%, wind 13 mph",Home,KAN,Kansas City Chiefs,Browns (deferred)
202101170nor,3750,20,1,1,1,0,0,189,33,18,2,199,7,30,35,1,127,17,8,31:31,1,316,0,62,10,"Sunday Jan 17, 2021",2021-01-17 18:40:00,3:12,20,0,0,1,1,3,190,35,20,2,190,5,20,25,0,104,11,6,28:29,0,294,4,34,0,NOR,New Orleans Saints,53.0 (under),Dome,Mercedes-Benz Superdome,Astroturf,6:40pm,New Orleans Saints -

In [11]:
boxscore = Boxscore('202009130min').dataframe
boxscore

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,...,roof,stadium,surface,time,vegas_line,weather,winner,winning_abbr,winning_name,won_toss
202009130min,None,31,2,1,1,0,0,364,44,32,...,Dome,U.S. Bank Stadium,Sportturf,1:00pm,Minnesota Vikings -1.0,None,Away,GNB,Green Bay Packers,Vikings (deferred)


In [18]:
boxscore

,attendance,away_first_downs,away_fourth_down_attempts,away_fourth_down_conversions,away_fumbles,away_fumbles_lost,away_interceptions,away_net_pass_yards,away_pass_attempts,away_pass_completions,away_pass_touchdowns,away_pass_yards,away_penalties,away_points,away_rush_attempts,away_rush_touchdowns,away_rush_yards,away_third_down_attempts,away_third_down_conversions,away_time_of_possession,away_times_sacked,away_total_yards,away_turnovers,away_yards_from_penalties,away_yards_lost_from_sacks,date,datetime,duration,home_first_downs,home_fourth_down_attempts,home_fourth_down_conversions,home_fumbles,home_fumbles_lost,home_interceptions,home_net_pass_yards,home_pass_attempts,home_pass_completions,home_pass_touchdowns,home_pass_yards,home_penalties,home_points,home_rush_attempts,home_rush_touchdowns,home_rush_yards,home_third_down_attempts,home_third_down_conversions,home_time_of_possession,home_times_sacked,home_total_yards,home_turnovers,home_yards_from_penalties,home_yards_lost_from_sacks,losing_abbr,losing_name,over_under,roof,stadium,surface,time,vegas_line,weather,winner,winning_abbr,winning_name,won_toss
202009130min,None,31,2,1,1,0,0,364,44,32,4,364,6,43,32,1,158,11,6,41:16,0,522,0,63,0,"Sunday Sep 13, 2020",2020-09-13 13:00:00,3:12,25,1,0,0,0,1,248,25,19,2,259,5,34,22,2,134,6,3,18:44,2,382,1,20,11,MIN,Minnesota Vikings,44.0 (over),Dome,U.S. Bank Stadium,Sportturf,1:00pm,Minnesota Vikings -1.0,None,Away,GNB,Green Bay Packers,Vikings (deferred)


In [9]:
boxscoreplayer = BoxscorePlayer(player_id=player_id, player_name=player_name, player_data=player_data)
df_boxscoreplayer = boxscoreplayer.dataframe

In [10]:
df_boxscoreplayer.T

,AdamDa01
completed_passes,None
attempted_passes,None
passing_yards,None
passing_touchdowns,None
interceptions_thrown,None
times_sacked,None
yards_lost_from_sacks,None
longest_pass,None
quarterback_rating,None
rush_attempts,None


# Roster

In [3]:
roster = Roster(team, year)
roster.coach, roster.players

Davante Adams (AdamDa01)
Montravius Adams (AdamMo00)
Jaire Alexander (AlexJa00)
Adrian Amos (AmosAd00)
Tavon Austin (AustTa00)
David Bakhtiari (BakhDa00)
Krys Barnes (BarnKr00)
Reggie Begelton (BegeRe00)
Henry Black (BlacHe00)
Tim Boyle (BoylTi00)
Ben Braden (BradBe00)
Hunter Bradley (BradHu00)
James Burgess (BurgJa01)
Oren Burks (BurkOr00)
Kenny Clark (ClarKe01)
Mason Crosby (CrosMa20)
Dominique Dafney (DafnDo00)
Josiah Deguara (DeguJo00)
AJ Dillon (DillAJ00)
Tyler Ervin (ErviTy00)
Tipa Galeai (GaleTi01)
Jonathan Garvin (GarvJo00)
Rashan Gary (GaryRa00)
Raven Greene (GreeRa02)
De'Jon Harris (HarrDe10)
Damon Harrison (HarrDa03)
Ka'dar Hollman (HollKa00)
Josh Jackson (JackJo01)
Elgton Jenkins (JenkEl00)
Aaron Jones (JoneAa00)
Kingsley Keke (KekeKi00)
Kevin King (KingKe01)
Christian Kirksey (KirkCh00)
Tyler Lancaster (LancTy00)
Allen Lazard (LazaAl00)
Marcedes Lewis (LewiMa00)
Corey Linsley (LinsCo00)
John Lovett (LoveJo02)
Dean Lowry (LowrDe00)
Kamal Martin (MartKa01)
Parry Nickerson (N

In [2]:
player = Player(player_id)
player

Davante Adams (AdamDa01)

In [8]:
player_data = Player(player_id).dataframe.to_html()

In [9]:
abstractplayer = AbstractPlayer(player_id=player_id, player_name=player, player_data=player_data)
abstractplayer

In [ ]:
abstractplayer

# Schedule

In [12]:
schedule = Schedule(team, year)
schedule

September 13 - MIN
September 20 - DET
September 27 - NOR
October 5 - ATL
October 18 - TAM
October 25 - HTX
November 1 - MIN
November 5 - SFO
November 15 - JAX
November 22 - CLT
November 29 - CHI
December 6 - PHI
December 13 - DET
December 19 - CAR
December 27 - OTI
January 3 - CHI
January 16 - RAM
January 24 - TAM

In [13]:
schedule[0]

September 13 - MIN

In [17]:
game = Game(schedule[0], game_type=REGULAR_SEASON, year=year)
# game

TypeError: 'Game' object is not callable

# Team

In [18]:
team = Team(team_name=team, year=year)
team

Green Bay Packers (GNB) - 2020

In [19]:
team.dataframe

,abbreviation,defensive_simple_rating_system,first_downs,first_downs_from_penalties,fumbles,games_played,interceptions,losses,margin_of_victory,name,...,rush_yards,rush_yards_per_attempt,simple_rating_system,strength_of_schedule,turnovers,win_percentage,wins,yards,yards_from_penalties,yards_per_play
GNB,GNB,1.8,358,28,6,16,5,3,8.8,Green Bay Packers,...,2118,4.8,7.7,-1.1,11,0.813,13,6224,684,6.3


In [20]:
teams = Teams(year=year)
teams

Green Bay Packers (GNB)
Buffalo Bills (BUF)
Tampa Bay Buccaneers (TAM)
Tennessee Titans (OTI)
New Orleans Saints (NOR)
Kansas City Chiefs (KAN)
Baltimore Ravens (RAV)
Seattle Seahawks (SEA)
Indianapolis Colts (CLT)
Las Vegas Raiders (RAI)
Minnesota Vikings (MIN)
Pittsburgh Steelers (PIT)
Arizona Cardinals (CRD)
Cleveland Browns (CLE)
Miami Dolphins (MIA)
Atlanta Falcons (ATL)
Dallas Cowboys (DAL)
Houston Texans (HTX)
Los Angeles Chargers (SDG)
Detroit Lions (DET)
San Francisco 49ers (SFO)
Los Angeles Rams (RAM)
Chicago Bears (CHI)
Carolina Panthers (CAR)
Washington Football Team (WAS)
Philadelphia Eagles (PHI)
New England Patriots (NWE)
Denver Broncos (DEN)
Cincinnati Bengals (CIN)
Jacksonville Jaguars (JAX)
New York Giants (NYG)
New York Jets (NYJ)

In [21]:
teams.dataframes

,abbreviation,defensive_simple_rating_system,first_downs,first_downs_from_penalties,fumbles,games_played,interceptions,losses,margin_of_victory,name,...,rush_yards,rush_yards_per_attempt,simple_rating_system,strength_of_schedule,turnovers,win_percentage,wins,yards,yards_from_penalties,yards_per_play
GNB,GNB,1.8,358,28,6,16,5,3,8.8,Green Bay Packers,...,2118,4.8,7.7,-1.1,11,0.813,13,6224,684,6.3
BUF,BUF,0.6,397,38,11,16,11,3,7.9,Buffalo Bills,...,1723,4.2,7.7,-0.2,22,0.813,13,6343,941,6.1
TAM,TAM,2.8,364,44,5,16,12,5,8.6,Tampa Bay Buccaneers,...,1519,4.1,9.4,0.8,17,0.688,11,6145,715,6.0
OTI,OTI,-3.1,381,36,5,16,7,5,3.3,Tennessee Titans,...,2690,5.2,1.6,-1.7,12,0.688,11,6343,783,6.2
NOR,NOR,4.5,367,21,9,16,8,4,9.1,New Orleans Saints,...,2265,4.6,9.6,0.5,17,0.750,12,6023,1005,5.8
KAN,KAN,2.3,397,32,9,16,7,2,6.9,Kansas City Chiefs,...,1799,4.5,6.8,-0.1,16,0.875,14,6653,919,6.3
RAV,RAV,4.3,327,19,7,16,11,5,10.3,Baltimore Ravens,...,3071,5.5,8.3,-2.0,18,0.688,11,5810,961,5.9
SEA,SEA,0.7,356,29,5,16,13,4,5.5,Seattle Seahawks,...,1971,4.8,5.5,0.0,18,0.750,12,5912,662,5.8
CLT,CLT,1.6,364,34,4,16,11,5,5.6,Indianapolis Colts,...,1996,4.3,2.8,-2.8,15,0.688,11,6049,899,5.9
RAI,RAI,-4.9,359,32,16,16,10,8,-2.8,Las Vegas Raiders,...,1916,4.2,-2.0,0.8,26,0.500,8,6133,856,5.9
